# ABS Quarterly National Accounts 5206

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Python-set-up" data-toc-modified-id="Python-set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Python set-up</a></span></li><li><span><a href="#Python-environment" data-toc-modified-id="Python-environment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Python environment</a></span></li><li><span><a href="#Get-data-from-ABS" data-toc-modified-id="Get-data-from-ABS-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get data from ABS</a></span></li><li><span><a href="#Plot" data-toc-modified-id="Plot-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot</a></span><ul class="toc-item"><li><span><a href="#Set-up-some-constants-we-will-use-across-all-plots" data-toc-modified-id="Set-up-some-constants-we-will-use-across-all-plots-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Set-up some constants we will use across all plots</a></span></li><li><span><a href="#Growth-charts---based-on-seasonally-adjusted-series" data-toc-modified-id="Growth-charts---based-on-seasonally-adjusted-series-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Growth charts - based on seasonally adjusted series</a></span></li><li><span><a href="#Four-quarter-rolling-totals---original-series" data-toc-modified-id="Four-quarter-rolling-totals---original-series-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Four quarter rolling totals - original series</a></span></li><li><span><a href="#Gross-Value-Added-(GVA)-by-Industry" data-toc-modified-id="Gross-Value-Added-(GVA)-by-Industry-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Gross Value Added (GVA) by Industry</a></span></li><li><span><a href="#Household-Final-Consumption-Expenditure" data-toc-modified-id="Household-Final-Consumption-Expenditure-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Household Final Consumption Expenditure</a></span></li><li><span><a href="#Taxes" data-toc-modified-id="Taxes-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Taxes</a></span></li><li><span><a href="#Government-Benefits-Payments" data-toc-modified-id="Government-Benefits-Payments-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>Government Benefits Payments</a></span></li><li><span><a href="#COVID-HFCE-comparison" data-toc-modified-id="COVID-HFCE-comparison-4.8"><span class="toc-item-num">4.8&nbsp;&nbsp;</span>COVID HFCE comparison</a></span></li></ul></li><li><span><a href="#Save-nominal-GDP-(it-is-useful-elsewhere)" data-toc-modified-id="Save-nominal-GDP-(it-is-useful-elsewhere)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Save nominal GDP (it is useful elsewhere)</a></span></li><li><span><a href="#Finished" data-toc-modified-id="Finished-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Finished</a></span></li></ul></div>

## Python set-up

In [1]:
# system imports
from pathlib import Path
import calendar
import re

# analytic imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# local imports
from python_env import python_env
from finalise_plot import finalise_plot
from abs_common import (get_fs_constants,
                        get_plot_constants,
                        get_ABS_meta_and_data,
                        get_identifier,
                        plot_Qgrowth,
                        NOMINAL_GDP_CSV 
                       )

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# plotting stuff
plt.style.use('ggplot')

In [2]:
cat_num = '5206'
CACHE_DIR, CHART_DIR, source = get_fs_constants(cat_num)

## Python environment

In [3]:
python_env()

--------------------------------------------------
System:     Darwin
Release:    20.1.0
Machine:    x86_64
Processor:  i386
RAM:        64GB
--------------------------------------------------
Python:     3.8.3
Psutil:     5.7.0
Pandas:     1.0.5
Numpy:      1.18.5
Matplotlib: 3.2.2
--------------------------------------------------


## Get data from ABS

In [4]:
url_template = ('https://www.abs.gov.au/statistics/economy/national-accounts/'
                'australian-national-accounts-national-income-expenditure-and-product/'
                'MONTH-YEAR/All_time_series_workbooks.zip')

meta, data = get_ABS_meta_and_data(url_template, CACHE_DIR)

File has been cached already
File for sep-2020 of size 5.4 MB


## Plot

### Set-up some constants we will use across all plots

In [5]:
RECENT, plot_times, plot_frames, plot_tags = get_plot_constants(data)

CVM = 'Chain volume measures'

### Growth charts - based on seasonally adjusted series

In [6]:
# identify available headline series from metadata
table = '1'
series_type = 'Seasonally Adjusted'
headline_plots = meta[
    (meta['Table'] == table) &
    (meta['Series Type'] == series_type) &
    (meta['Unit'] == '$ Millions') &
    (meta['Data Item Description'].str.contains(CVM))    
]['Data Item Description']

for description in headline_plots:
    
    # identify data
    series, _ = get_identifier(meta, description,
                               series_type, table)
    
    # plot
    for plot_from, plot_tag in zip(plot_times, plot_tags):

        title = description.replace(' ;', '')
        
        plot_Qgrowth(data[series].copy(), 
            title, plot_from, 
            plot_tag, CHART_DIR,
            rfooter=f'{source} {table}',
            lfooter=f'From the {series_type.lower()} series')

### Four quarter rolling totals - original series

In [7]:
# uses same series from previous cell
series_type = "Original"
for description in headline_plots:

    series, units = get_identifier(meta, description,
                               series_type, table)
    rolling4Q = data[series].rolling(4).sum()
    units = f'{units} / Year'
    
    for plot_time, plot_tag in zip(plot_times, plot_tags):
        plot_data = rolling4Q
        if plot_time:
            plot_data = rolling4Q[rolling4Q.index >= plot_time]
        ax = plot_data.plot(lw=2, color="#dd0000")
        
        title = description.replace(' ;', '')
        
        finalise_plot(ax, title, units, plot_tag, CHART_DIR,
            rfooter=f'{source} {table}',
            lfooter=f'4Q rolling sum {series_type.lower()} series'
        )

### Gross Value Added (GVA) by Industry

In [8]:
# identify available series from metadata
table = '6'
series_type = 'Seasonally Adjusted'
industries = meta[
    (meta['Table'] == table) &
    (meta['Series Type'] == series_type) &
    (meta['Unit'] == '$ Millions') # limit to money
]['Data Item Description']
industry_start = pd.Timestamp('1974-09-01')

for industry in industries:

    # get data
    series, units = get_identifier(meta, industry,
                               series_type, table)
    units = f'{units}/Qtr'
    
    # plot
    for plot_frame, plot_tag in zip(plot_frames, plot_tags):
        used_frame = plot_frame[plot_frame.index >= industry_start]
        ax = used_frame[series].plot(lw=2, color="#dd0000")

        # we limit subgroup titles to keep the width
        # within what would fit on the top of a plot
        title = industry
        if not title.endswith(") ;"):
            title = re.sub("^[^\(]+\(", "(", title)
        title = f' GVA: {title}'

        finalise_plot(ax, title, units, plot_tag, CHART_DIR,
            rfooter=f'{source} {table}',
            lfooter=(f'{series_type.capitalize()}, '
                     f'{CVM.lower()}')
        )

###  Household Final Consumption Expenditure

In [9]:
# identify available series from metadata
table = '8'
series_type = 'Seasonally Adjusted'
hfce = meta[
    (meta['Table'] == table) &
    (meta['Series Type'] == series_type) &
    (meta['Unit'] == '$ Millions') & # limit to money
    (meta['Data Item Description'].str.contains(CVM))
]['Data Item Description']

for description in hfce:
    
    # get data
    series, units = get_identifier(meta, description,
                                   series_type, table)
    units = f'{units}/Qtr'
    
    # plot
    for plot_frame, plot_tag in zip(plot_frames, plot_tags):
        ax = plot_frame[series].plot(lw=2, color="#dd0000")
        
        title = description.replace(f': {CVM} ;', '')
        title = f'Household spending: {title}'
        
        finalise_plot(ax, title, units, plot_tag, CHART_DIR,
            rfooter=f'{source} {table}',
            lfooter=(f'{series_type.capitalize()}, '
                     f'{CVM.lower()}')
        )

### Taxes

In [10]:
# identify available series from metadata
table = '22'
series_type = 'Seasonally Adjusted'
tax_plots = meta[
    (meta['Table'] == table) &
    (meta['Series Type'] == series_type) 
]['Data Item Description']

for plot in tax_plots:
    
    # data capture
    ident, units = get_identifier(meta, plot, series_type, table)
    units = f'{units}/Qtr'
    series = data[ident]
    series = series[series.index >= RECENT]
    if series.isna().all():
        # don't plot empty series
        continue
    
    # plot
    ax = series.plot(lw=2, color="#dd0000")
    title = plot.replace(' ;', '')
    finalise_plot(ax, title, 
                  f'{units} (Current Prices)', 
                  'govt-tax', CHART_DIR,
                  rfooter=f'{source} {table}',
                  lfooter=f'{series_type.capitalize()} series')

### Government Benefits Payments

In [11]:
table = '23'
series_type = 'Original'
plots_payments = meta[
    (meta['Table'] == table) &
    (meta['Series Type'] == series_type) 
]['Data Item Description']

for plot in plots_payments:
    
    # get data
    ident, units = get_identifier(meta, plot, series_type, table)
    units = f'{units}/Qtr'
    series = data[ident]
    series = series[series.index >= RECENT]
    if series.isna().all():
        # don't plot empty series
        continue

    # plot
    ax = series.plot(lw=2, color="#dd0000")
    
    # - from ABS description to plot title
    title = plot.replace('General government - National ;  ',
                         'Federal Govt - ')
    title = title.replace('General government - State and local ;  ',
                         'State or Local Govt - ')
    title = title.replace('General government ;  ',
                         'All Govt - ')
    title = title.replace(' ;', '')
    
    finalise_plot(ax, title, f'{units} (Current Prices)', 
                  'govt-spending', CHART_DIR,
                  rfooter=f'{source} {table}',
                  lfooter=f'{series_type.capitalize()} series')
        

### COVID HFCE comparison

In [12]:
# used for all COVID indexes
REFERENCE = pd.Timestamp('2019-12-01')
ref_string = f'{calendar.month_abbr[REFERENCE.month]} {REFERENCE.year}'
index_base = 100
df = data[data.index >= RECENT]
series_type = "Seasonally Adjusted"

In [13]:
# HFCE component of state final demand 
# Victoria compared with the rest of Australia

# identify the tables that contain the detailed state HFCE data
states = {
    # state: table
    'NSW': '26',
    'Vic': '27',
    'Qld': '28',
    'SA':  '29',
    'WA':  '30',
    'Tas': '31',
    'NT':  '32',
    'ACT': '33',
}

# identify available series from metadata
HFCE = 'Households ;  Final consumption expenditure - '
plots = meta.loc[
    meta['Data Item Description'].str.contains(HFCE)
    & meta['Data Item Description'].str.contains(CVM)
    & ~meta['Data Item Description'].str.contains('Revisions')
    & (meta['Table'] == states['Vic'])
    & (meta['Series Type'] == series_type)
]['Data Item Description']

for plot in plots:
    # get the data
    rest = []
    for state in states:
        series, units = get_identifier(meta, plot,
                                       series_type, 
                                       states[state])
        if state == 'Vic':
            victoria = df[series]
        else: 
            rest.append(series)
    rest = df[rest].sum(axis=1)
    
    # plot
    plotable = pd.DataFrame([victoria, rest]).T
    plotable.columns = ['Victoria', 'Rest of Australia']
    reference_row = plotable.loc[REFERENCE]
    plotable = plotable.div(reference_row) * index_base
    title = plot.replace(HFCE, '')
    title = title.replace(f': {CVM} ;', '')
    title = f'Household spending: {title}'

    ax = plotable.plot(lw=2, color=['darkorange', 'blue'])
    finalise_plot(ax, title, 
                  f'Index ({ref_string} = {index_base})', 
                  'COVID', CHART_DIR,
                  rfooter=f'{source}s {", ".join(states.values())}',
                  lfooter=f'{series_type.capitalize()}, '
                           f'{CVM.lower()}'
                 )

## Save nominal GDP (it is useful elsewhere)

In [14]:
table = '24' # Selected Analytical Series
series_type = 'Original'
descriptions = [
    'Non-farm ;  Gross domestic product: Current prices ;',
    'Farm ;  Gross domestic product: Current prices ;',
]
frame = {}
for series in descriptions:
    ident, units = get_identifier(meta, series, series_type, table)
    frame[series] = data[ident]
print(f'From table {table}: {units}')
frame = pd.DataFrame(frame)
frame['Total quarterly'] = frame.sum(axis=1)
frame['Rolling Annual'] = frame['Total quarterly'].rolling(4).sum()

table = '34' # Key Aggregates and analytical series, Annual
series = 'Gross domestic product: Current prices ;'
ident, units = get_identifier(meta, series, series_type, table)
print(f'From table {table}: {units}')
frame[f'Annual {series}'] =  data[ident]

frame.to_csv(NOMINAL_GDP_CSV)

From table 24: $ Millions
From table 34: $ Millions


## Finished

In [15]:
print('Finished')

Finished
